In [ ]:
%pip install scipy
%pip install ripser gudhi
%pip install scikit-tda
%pip install numpy
%pip install tqdm
%pip install networkx
%pip install persim
%pip install pot
%pip install seaborn
%pip install matplotlib
%pip install pandas
%pip install gudhi
%pip install mayavi

# Before Run, properly set the following variables:

In [ ]:
geodesic_data_path = "data/preprocessing/geodesic/" # path to preprocessed geodesic data
euclidean_data_path = "data/preprocessing/euclidean/" # path to preprocessed euclidean data

## 1. Preprocessing

In [ ]:
from src.preprocessing.preprocessing import *
import numpy as np
from scipy.io import loadmat
import re

def load_data_from_mat(mat_dir):
    mats = []
    X = []
    y = []

    for file in os.listdir(mat_dir):
        if file.endswith(".mat"):
            mats.append(file)
    mats.sort()

    pattern = r'_([a-zA-Z]+)\d+\.mat'
    for i in range(len(mats)):        
        mat = loadmat(mat_dir + mats[i])
        X.append(mat["dm"])
        class_name = re.search(pattern, mats[i])
        
        if class_name:
            y.append(class_name.group(1))
            print(f"added {class_name.group(1)}")
        
    X = np.stack(X)
    return X, np.array(y)

VR persistence diagram with Ripser

In [ ]:
import os
from src.shape.shape import *
from tqdm.notebook import tqdm
# Load preprocessed data


shape_paths = []
for file in os.listdir(geodesic_data_path):
    if file.endswith(".mat"):
        shape_paths.append(geodesic_data_path + file)
shape_paths.sort()
print(f"Found {len(shape_paths)} shapes... {shape_paths[0], shape_paths[1]}, ...")

shapes = []
for f in tqdm(shape_paths, desc="Loading shapes"):
    shapes.append(load_from_mat(f))
    
print(f"Loaded {len(shapes)} shapes... {shapes[0], shapes[1]}, ...")


# Load Euclidean Preprocessed data
euclidean_shape_paths = []
for file in os.listdir(euclidean_data_path):
    if file.endswith(".mat"):
        euclidean_shape_paths.append(euclidean_data_path + file)

euclidean_shape_paths.sort()
print(f"Found {len(euclidean_shape_paths)} shapes... {euclidean_shape_paths[0], euclidean_shape_paths[1]}, ...")

euclidean_shapes = []
for f in tqdm(euclidean_shape_paths, desc="Loading shapes"):
    euclidean_shapes.append(load_from_mat(f))

### Visualize

In [ ]:
import matplotlib.pyplot as plt
from mayavi import mlab

def show_shape(shape):
    x = shape.coordinates[:, 0]
    y = shape.coordinates[:, 1]
    z = shape.coordinates[:, 2]

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x, y, z, c=z, cmap='viridis')
    plt.show()

def show_shape_interactive(shape):
    x = shape.coordinates[:, 0]
    y = shape.coordinates[:, 1]
    z = shape.coordinates[:, 2]

    mlab.figure(bgcolor=(0, 0, 0))  # You can choose a background color
    points = mlab.points3d(x, y, z, z, colormap='viridis', scale_mode='none', scale_factor=1.2)
    mlab.axes()
    mlab.show()




Visualize shapes

In [ ]:
shape_euc_0 = euclidean_shapes[0]

show_shape_interactive(shape_euc_0)

shape_0 = shapes[0]

show_shape_interactive(shape_0)

In [ ]:
# Load distance matrix and labels
import numpy as np
from scipy.io import loadmat
import re

def load_data_from_mat(mat_dir):
    mats = []
    X = []
    y = []

    for file in os.listdir(mat_dir):
        if file.endswith(".mat"):
            mats.append(file)
    mats.sort()

    pattern = r'_([a-zA-Z]+)\d+\.mat'
    for i in range(len(mats)):        
        mat = loadmat(mat_dir + mats[i])
        X.append(mat["dm"])
        class_name = re.search(pattern, mats[i])
        
        if class_name:
            y.append(class_name.group(1))
            # print(f"added {class_name.group(1)}")
        
    X = np.stack(X)
    return X, np.array(y)

euclidean_dataset_dir = "data/preprocessing/euclidean/"
geodesic_dataset_dir = "data/preprocessing/geodesic/"

X_euclidean, y_euclidean = load_data_from_mat(euclidean_dataset_dir)
X_geodesic, y_geodesic = load_data_from_mat(geodesic_dataset_dir)

print(f"X_euclidean shape: {X_euclidean.shape}")
print(f"X_geodesic shape: {X_geodesic.shape}")

print(f"y_euclidean shape: {len(y_euclidean)}")
print(f"y_geodesic shape: {len(y_geodesic)}")


In [ ]:
import pandas as pd
from matplotlib.patches import Rectangle
from matplotlib import pyplot as plt

y_df = pd.DataFrame({"y": y_geodesic})
num_classes = y_df['y'].nunique()
print(f"Found {num_classes} classes")

def plot_distance_matrix(distance_matrix, distance_name):
    plt.imshow(distance_matrix)

    start = 0
    curr=0
    shape_classes = []
    mean_between_cluster_distances = []
    mean_within_cluster_distances = []
    ratio_within_between_cluster_distances = []
    std_between_cluster_distances = []
    std_within_cluster_distances = []
    for i in range(len(y_df)):
        if y_df['y'][i] != y_df['y'][start]:
            box_x = start
            box_y = start
            box_width = i - start
            box_height = i - start
            box = Rectangle((box_x , box_y ), box_width, box_height, fill=False, color='red', linewidth=1)
            plt.gca().add_patch(box)
            
            
            between_cluster_distances = np.concatenate([distance_matrix[start:i, :i], distance_matrix[start:i, i:]], axis = 1)
            mean_between_cluster_distance = np.mean(between_cluster_distances)
            std_between_cluster_distance = np.std(between_cluster_distances)
            within_cluster_distances = distance_matrix[start:i, start:i]
            mean_within_cluster_distance = np.mean(within_cluster_distances)
            std_within_cluster_distance = np.std(within_cluster_distances)

            ratio_within_between_cluster_distance = mean_within_cluster_distance / mean_between_cluster_distance
            print(f"Shape: {y_df['y'][start]}, mean_between_cluster_distance: {mean_between_cluster_distance}, mean_within_cluster_distance: {mean_within_cluster_distance}, ratio wcd/bcd: {ratio_within_between_cluster_distance},std_between_cluster_distance: {std_between_cluster_distance}, std_within_cluster_distance: {std_within_cluster_distance}")
            
            shape_classes.append(y_df['y'][start])
            mean_between_cluster_distances.append(mean_between_cluster_distance)
            mean_within_cluster_distances.append(mean_within_cluster_distance)
            std_between_cluster_distances.append(std_between_cluster_distance)
            std_within_cluster_distances.append(std_within_cluster_distance)
            ratio_within_between_cluster_distances.append(ratio_within_between_cluster_distance)
            
            start = i

    box_x = start
    box_y = start
    box_width = len(y_df) - start
    box_height = len(y_df) - start
    box = Rectangle((box_x , box_y ), box_width, box_height, fill=False, color='red', linewidth=1)
    plt.gca().add_patch(box)

    plt.colorbar()
    plt.title(distance_name + " matrix")
    plt.savefig(distance_name + "_matrix.png")
    plt.show()

    np.savetxt("data/" +distance_name+"_heatmap.csv", distance_matrix, delimiter=",")


    distance_mean_df = pd.DataFrame({"shape_class": shape_classes, 
    "mean_BCD": mean_between_cluster_distances, 
    "mean_WCD": mean_within_cluster_distances,
    "ratio_WCD_BCD": ratio_within_between_cluster_distances,
    "std_BCD": std_between_cluster_distances,
    "std_WCD": std_within_cluster_distances})
    distance_mean_df.to_csv(f"data/_{distance_name}_mean_distances.csv")
    return distance_mean_df


def report_distance_matrix(distance_mean_df, distance_name):        
    plt.figure(figsize=(10, 6))
    plt.bar(distance_mean_df['shape_class'], distance_mean_df['mean_BCD'], 0.2,label='Between Cluster Distance', alpha=0.7,)
    plt.bar(distance_mean_df['shape_class'], distance_mean_df['mean_WCD'], 0.2, label='Within Cluster Distance', alpha=0.7)

    plt.xlabel('Shape Class')
    plt.ylabel('mean Distance')
    plt.title(f'Mean Between and Within Cluster Distances by Shape Class: {distance_name}')
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()

    # Show the plot
    plt.show()
    print(distance_mean_df)




In [ ]:
from tqdm.notebook import tqdm
dDiam_matrix = np.zeros((len(shapes), len(shapes)))


tqdm_bar = tqdm(total=len(shapes)**2 / 2, desc = "Computing dDiam matrix")
for i in range(len(shapes)):
    for j in range(i+1, len(shapes)):
        dDiam_matrix[i,j] = dDiam(shapes[i], shapes[j])
        dDiam_matrix[j,i] = dDiam_matrix[i,j]
        tqdm_bar.update(1)

tqdm_bar.close()
# save to csv
np.savetxt("data/dDiam_distance_matrix.csv", dDiam_matrix, delimiter=",")


In [ ]:
# plot dDiam matrix
import matplotlib.pyplot as plt
mean_df = plot_distance_matrix(dDiam_matrix, "dDiam")
report_distance_matrix(mean_df, "dDiam")

In [ ]:
d_E_inf_matrix = np.zeros((len(shapes), len(shapes)))

tqdm_bar = tqdm(total=len(shapes)**2 / 2, desc = "Computing d_E_inf matrix")
for i in range(len(shapes)):
    for j in range(i+1, len(shapes)):
        d_E_inf_matrix[i,j] = d_E_inf(euclidean_shapes[i], euclidean_shapes[j])
        d_E_inf_matrix[j,i] = d_E_inf_matrix[i,j]
        tqdm_bar.update(1)


tqdm_bar.close()
# save to csv
np.savetxt("data/d_E_inf_distance_matrix.csv", d_E_inf_matrix, delimiter=",")

In [ ]:
# plot distance matrix

mean_df = plot_distance_matrix(d_E_inf_matrix, "d_E_inf")
report_distance_matrix(mean_df, "d_E_inf")

In [ ]:
d_G_1_matrix = np.zeros((len(shapes), len(shapes)))
d_G_inf_matrix = np.zeros((len(shapes), len(shapes)))

tqdm_bar = tqdm(total=len(shapes)**2 / 2, desc = "Computing d_G_1 and g_G_inf matrix")
for i in range(len(shapes)):
    for j in range(i+1, len(shapes)):
        d_G_1_matrix[i,j] = d_G_wasserstein(shapes[i], shapes[j], q=1)
        d_G_inf_matrix[i,j] = d_G_wasserstein(shapes[i], shapes[j], q=np.inf)
        d_G_1_matrix[j,i] = d_G_1_matrix[i,j]
        d_G_inf_matrix[j,i] = d_G_inf_matrix[i,j]
        tqdm_bar.update(1)


tqdm_bar.close()
# save to csv
np.savetxt("data/d_G_1_distance_matrix.csv", d_G_1_matrix, delimiter=",")
np.savetxt("data/d_G_inf_distance_matrix.csv", d_G_inf_matrix, delimiter=",")


In [ ]:
mean_df_d_G_1 = plot_distance_matrix(d_G_1_matrix, "d_G_1")
mean_df_d_G_inf = plot_distance_matrix(d_G_inf_matrix, "d_G_inf")

report_distance_matrix(mean_df_d_G_1, "d_G_1")
report_distance_matrix(mean_df_d_G_inf, "d_G_inf")

In [ ]:
# compute the mean diameter of a class
import numpy as np
def mean_diameter_class(shapes, class_label, y_labels):
    indices = np.where(y_labels == class_label)[0]
    # print(indices)
    # shapes_to_see = shapes[*indices]
    diams = []
    for i in indices:
        diams.append(np.max(shapes[i].dm))
    return np.mean(diams)



In [ ]:
class_to_see = "horse"
print(f"mean diam of class {class_to_see} is {mean_diameter_class(shapes, class_to_see, y_geodesic)}")
class_to_see = "seahorse"
print(f"mean diam of class {class_to_see} is {mean_diameter_class(shapes, class_to_see, y_geodesic)}")

# Mean diameter for each class

In [ ]:
unique_classes = np.unique(y_geodesic)

mean_diams = []
for c in unique_classes:
    mean_diams.append(mean_diameter_class(shapes, c, y_geodesic))


print(unique_classes)
print(mean_diams)

from matplotlib import pyplot as plt

plt.figure(figsize=(10,6))
plt.bar(unique_classes, mean_diams, color = "skyblue")
plt.xlabel("Classes")
plt.ylabel("mean diameter")
plt.title("Mean diameter of each class (geodesic)")
plt.xticks(rotation=45)
plt.show()


In [ ]:
unique_classes = np.unique(y_geodesic)

mean_diams = []
for c in unique_classes:
    mean_diams.append(mean_diameter_class(euclidean_shapes, c, y_euclidean))


print(unique_classes)
print(mean_diams)

from matplotlib import pyplot as plt

plt.figure(figsize=(10,6))
plt.bar(unique_classes, mean_diams, color = "skyblue")
plt.xlabel("Classes")
plt.ylabel("mean diameter")
plt.title("Mean diameter of each class (Euclidean)")
plt.xticks(rotation=45)
plt.show()


### Dendrogram: SLHC

In [ ]:
from scipy.cluster.hierarchy import single, dendrogram, complete
from scipy.spatial.distance import squareform

X_geodesic, y_geodesic = load_data_from_mat(geodesic_dataset_dir)
X_euclidean, y_euclidean = load_data_from_mat(euclidean_dataset_dir)

converted_d_G_1_matrix = squareform(d_G_1_matrix, checks= False)
converted_d_G_inf_matrix = squareform(d_G_inf_matrix, checks= False)
converted_d_E_inf_matrix = squareform(d_E_inf_matrix, checks= False)

linked_d_G_1 = single(converted_d_G_1_matrix)
linked_d_G_inf = single(converted_d_G_inf_matrix)
linked_d_E_inf = single(converted_d_E_inf_matrix)

plt.figure(0, figsize=(20, 7))
dendrogram(linked_d_G_1, orientation='top', distance_sort='descending', show_leaf_counts=False, labels=y_geodesic, leaf_font_size=10)
plt.title("d_G_1 dendrogram")
plt.xlabel("Shape index")
plt.ylabel("Distance")
plt.savefig("d_G_1_dendrogram.png")
plt.show()

plt.figure(1, figsize=(20, 7))
dendrogram(linked_d_G_inf, orientation='top', distance_sort='descending', show_leaf_counts=False, labels=y_geodesic, leaf_font_size=10)
plt.title("d_G_inf dendrogram")
plt.xlabel("Shape index")
plt.ylabel("Distance")
plt.savefig("d_G_inf_dendrogram.png")
plt.show()

plt.figure(2, figsize=(20, 7))
dendrogram(linked_d_E_inf, orientation='top', distance_sort='descending', show_leaf_counts=False, labels = y_euclidean, leaf_font_size=10)
plt.title("d_E_inf dendrogram")
plt.xlabel("Shape index")
plt.ylabel("Distance")
plt.savefig("d_E_inf_dendrogram.png")
plt.show()


### Complete Linkage

In [ ]:
from scipy.cluster.hierarchy import single, dendrogram, complete
from scipy.spatial.distance import squareform

linked_d_G_1 = complete(converted_d_G_1_matrix)
linked_d_G_inf = complete(converted_d_G_inf_matrix)
linked_d_E_inf = complete(converted_d_E_inf_matrix)

plt.figure(0, figsize=(20, 7))
dendrogram(linked_d_G_1, orientation='top', distance_sort='descending', show_leaf_counts=False, labels=y_geodesic, leaf_font_size=10)
plt.title("d_G_1 dendrogram")
plt.xlabel("Shape index")
plt.ylabel("Distance")
plt.savefig("d_G_1_dendrogram.png")
plt.show()

plt.figure(1, figsize=(20, 7))
dendrogram(linked_d_G_inf, orientation='top', distance_sort='descending', show_leaf_counts=False, labels=y_geodesic, leaf_font_size=10)
plt.title("d_G_inf dendrogram")
plt.xlabel("Shape index")
plt.ylabel("Distance")
plt.savefig("d_G_inf_dendrogram.png")
plt.show()

plt.figure(2, figsize=(20, 7))
dendrogram(linked_d_E_inf, orientation='top', distance_sort='descending', show_leaf_counts=False, labels = y_euclidean, leaf_font_size=10)
plt.title("d_E_inf dendrogram")
plt.xlabel("Shape index")
plt.ylabel("Distance")
plt.savefig("d_E_inf_dendrogram.png")
plt.show()

## Multidimensional scaling Plot

In [ ]:
import seaborn as sns
from sklearn.manifold import MDS

mds_d_G_1 = MDS(n_components=2, dissimilarity="precomputed", random_state=0)
mds_d_G_inf = MDS(n_components=2, dissimilarity="precomputed", random_state=0)
mds_d_E_inf = MDS(n_components=2, dissimilarity="precomputed", random_state=0)
mds_d_Diam = MDS(n_components=2, dissimilarity="precomputed", random_state=0)

mds_result_d_G_1 = mds_d_G_1.fit_transform(d_G_1_matrix, )
mds_result_d_G_inf = mds_d_G_inf.fit_transform(d_G_inf_matrix)
mds_result_d_E_inf = mds_d_E_inf.fit_transform(d_E_inf_matrix)
mds_result_d_Diam = mds_d_Diam.fit_transform(dDiam_matrix)

colors = sns.color_palette("bright", len(np.unique(y_geodesic)))

plt.figure(0, figsize=(15, 7))
for label in np.unique(y_geodesic):
    indices = np.where(y_geodesic == label)[0]
    plt.scatter(mds_result_d_G_1[indices,0], mds_result_d_G_1[indices,1], label = label, marker = 'o', )
plt.title("d_G_1 MDS")
plt.legend()
plt.show()

plt.figure(1, figsize=(15, 7))
for label in np.unique(y_geodesic):
    indices = np.where(y_geodesic == label)[0]
    plt.scatter(mds_result_d_G_inf[indices,0], mds_result_d_G_inf[indices,1], label = label, marker = 'o', )
plt.title("d_G_inf MDS")
plt.legend()
plt.show()

plt.figure(2, figsize=(15, 7))
for label in np.unique(y_euclidean):
    indices = np.where(y_euclidean == label)[0]
    plt.scatter(mds_result_d_E_inf[indices,0], mds_result_d_E_inf[indices,1], label = label, marker = 'o', )
plt.title("d_E_inf MDS")
plt.legend()
plt.show()

plt.figure(3, figsize=(15, 7))
for label in np.unique(y_geodesic):
    indices = np.where(y_geodesic == label)[0]
    plt.scatter(mds_result_d_Diam[indices,0], mds_result_d_Diam[indices,1], label = label, marker = 'o', )
plt.title("d_Diam MDS")
plt.legend()
plt.show()



### Classification using dDiam

In [ ]:
# 1. dendrogram

from scipy.cluster.hierarchy import single, dendrogram
from scipy.spatial.distance import squareform

converted_d_diam_matrix = squareform(dDiam_matrix, checks= False)

linked_d_diam = complete(converted_d_diam_matrix)

plt.figure(0, figsize=(15, 7))
dendrogram(linked_d_diam, orientation='top', distance_sort='descending', show_leaf_counts=False, labels=y_geodesic)
plt.title("dDiam dendrogram")

plt.xlabel("Shape index")
plt.ylabel("Distance")
plt.savefig("dDiam_dendrogram.png")
plt.show()

In [ ]:
# 2. k-means with exact K-value (k=12)
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, adjusted_rand_score

def evaluate_clustering(distance_matrix, name,y_labels, k=12):
    kmeans = KMeans(n_clusters=k, random_state=0).fit(distance_matrix)
    labels = kmeans.labels_

    y_encoded = LabelEncoder().fit_transform(y_labels)
    sil = silhouette_score(distance_matrix, labels, metric='precomputed')
    rand = adjusted_rand_score(y_encoded, labels)
    print(f"{name} - Silhouette score: {sil}")
    print(f"{name} - Adjusted rand score: {rand}")

    return labels, sil, rand


## Compare Euclidean distance and Geodesic distance

In [ ]:

y_geodesic_encoded = LabelEncoder().fit_transform(y_geodesic)
y_euclidean_encoded = LabelEncoder().fit_transform(y_euclidean)

labels_dDiam, sil_diam, rand_diam = evaluate_clustering(dDiam_matrix, "dDiam", y_geodesic, k=12)
labels_d_G_1, sil_d_G_1, rand_d_G_1 = evaluate_clustering(d_G_1_matrix, "d_G_1", y_geodesic, k=12)
labels_d_G_inf, sil_d_G_inf, rand_d_G_inf = evaluate_clustering(d_G_inf_matrix, "d_G_inf", y_geodesic, k=12)
labels_d_E_inf, sil_d_E_inf, rand_d_E_inf = evaluate_clustering(d_E_inf_matrix, "d_E_inf", y_euclidean, k=12)

plt.figure(0, figsize=(15, 7))
# bar plot. sil_diam rand_diam  sil_d_G_1 rand_d_G_1 sil_d_G_inf rand_d_G_inf sil_d_E_inf rand_d_E_inf
plt.bar(["dDiam", "d_G_1", "d_G_inf", "d_E_inf"], [sil_diam, sil_d_G_1, sil_d_G_inf, sil_d_E_inf], color = "skyblue")
plt.xlabel("Distance matrix")
plt.ylabel("Silhouette score")
plt.title("Silhouette score of each distance matrix")
plt.xticks(rotation=45)
plt.show()

cluster_eval_df = pd.DataFrame({"distance_matrix": ["dDiam", "d_G_1", "d_G_inf", "d_E_inf"],
                                "silhouette_score": [sil_diam, sil_d_G_1, sil_d_G_inf, sil_d_E_inf],
                                "adjusted_rand_score": [rand_diam, rand_d_G_1, rand_d_G_inf, rand_d_E_inf]})


# Reshape for plotting
df_melted = cluster_eval_df.melt(id_vars='distance_matrix', var_name='Score_Type', value_name='Score')

# Plot
plt.figure(figsize=(10, 6))
sns.barplot(x='distance_matrix', y='Score', hue='Score_Type', data=df_melted)

# Customizing the plot
plt.xlabel('Distance Metric')
plt.ylabel('Score')
plt.title('Comparison of Silhouette Scores and Adjusted Rand Scores')
plt.legend(title='Score Type')

# Show the plot
plt.show()
